In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

c:\Users\luisa\anaconda3\envs\aifb\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
def pt_file_to_tensor(file):
    """
    Converts a file to a tensor
    :param file: the file to be converted
    :return: the tensor
    """
    with open(file, 'rb') as f:
        tensor = torch.load(f)
    return tensor

pred = pt_file_to_tensor('out/IMDB/RGAT_no_emb/pred_before.pt')
pred_after_2597 = pt_file_to_tensor('out/IMDB/RGAT_no_emb/pred_after(1, tensor(2597))adapt_tensor([   2, 2082])_5.pt')
pred_after_673 = pt_file_to_tensor('out/IMDB/RGAT_no_emb/pred_after(2, tensor(673))adapt_1894_6.pt')
pred_after_1894 = pt_file_to_tensor('out/IMDB/RGAT_no_emb/pred_after(2, tensor(673))adapt_tensor([   3, 1894])_9.pt')
pred_after_2657 = pt_file_to_tensor('out/IMDB/RGAT_no_emb/pred_after(3, tensor(2657))adapt_780_0.pt')
pred_after_780 = pt_file_to_tensor('out/IMDB/RGAT_no_emb/pred_after(3, tensor(2657))adapt_tensor([  2, 780])_17.pt')
pred



tensor([[2.4940e-01, 2.4994e-01, 2.5009e-01, 2.5058e-01],
        [2.5000e-01, 2.5000e-01, 2.5000e-01, 2.5000e-01],
        [2.4963e-01, 2.4989e-01, 2.4997e-01, 2.5052e-01],
        ...,
        [2.4959e-01, 2.5034e-01, 2.4957e-01, 2.5049e-01],
        [2.4957e-01, 2.5027e-01, 2.4969e-01, 2.5046e-01],
        [1.8273e-04, 4.9089e-07, 9.9965e-01, 1.6961e-04]])

In [21]:
pred_after_2597

tensor([[2.4940e-01, 2.4994e-01, 2.5009e-01, 2.5058e-01],
        [2.5000e-01, 2.5000e-01, 2.5000e-01, 2.5000e-01],
        [2.4963e-01, 2.4989e-01, 2.4997e-01, 2.5052e-01],
        ...,
        [2.4959e-01, 2.5034e-01, 2.4957e-01, 2.5049e-01],
        [2.4957e-01, 2.5027e-01, 2.4969e-01, 2.5046e-01],
        [1.8273e-04, 4.9089e-07, 9.9965e-01, 1.6961e-04]])

In [23]:
(pred_after_780-pred).to_sparse_coo()

tensor(indices=tensor([], size=(2, 0)),
       values=tensor([], size=(0,)),
       size=(2929, 4), nnz=0, layout=torch.sparse_coo)

In [26]:
(pred-pred_after_1894).to_sparse_coo()

tensor(indices=tensor([], size=(2, 0)),
       values=tensor([], size=(0,)),
       size=(2929, 4), nnz=0, layout=torch.sparse_coo)

In [27]:
pd.read_csv('out/IMDB/RGAT_no_emb/LRP_nodes_table.csv')

,Unnamed: 0,Unnamed: 1,tensor_nodes,max_nodes,pos_max_nodes,min_nodes,pos_min_nodes
0,0,tensor(341),"tensor([0., 0., 0., ..., 0., 0., 0.])",0.000000,0,0.000000,0
1,1,tensor(2597),"tensor([0., 0., 0., ..., 0., 0., 0.])",0.161573,1582,-0.000882,639
2,2,tensor(673),"tensor([0., 0., 0., ..., 0., 0., 0.])",3.555420,1894,-5.212372,780
3,3,tensor(2657),"tensor([0., 0., 0., ..., 0., 0., 0.])",8.058411,780,-9.297858,2657
4,4,tensor(865),"tensor([0., 0., 0., ..., 0., 0., 0.])",0.824438,2480,-0.022616,865
